# Tarea 2
### Integrantes:
- Pablo Araya
- Benjamin Barrientos
- Sebastian Lopez

# P1. Carga y transformación de datos
Al abordar un problema de clasificacióon de imágenes con redes neuronales profundas, es importante optimizar
el proceso de carga de datos. Al tratarse de im´agenes, es de esperar que no todas las muestras puedan ser
almacenadas en memoria de manera simultanea, por lo que es necesario construir un generador. El objetivo de la
sección es construir un generador de muestras eficiente, que minimice el tiempo de carga de datos y optimice el
uso de memoria. 

El conjunto de datos a utilizar esta disponible en este link. En la Figura 1 se observan algunas muestras
de cada clase. La carpeta train consta de alrededor de 5.000 im´agenes y la carpeta test, cerca de 600. Al ser
un conjunto de datos peque˜no, se emplean t´ecnicas de aumentaci´on de datos que ser´an detalladas a continuaci´on.

In [1]:
import torch

# Se asigna un valor de reproductibilidad
torch.manual_seed(6202)

In [2]:
import pandas as pd
import numpy as np
from torchvision import transforms
import torchvision.datasets as datasets
import PIL
import matplotlib.pyplot as plt

In [167]:
def loader(path):
    Img = PIL.Image.open(path)
    if len(Img.split()) == 1:
        im = PIL.Image.merge("RGB", (Img.split()[0], Img.split()[0], Img.split()[0]))
        return im
    else:
        return Img

transformaciones = transforms.Compose([transforms.Resize([224,224]),
                                       transforms.ColorJitter(brightness=(0,1)),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomRotation(degrees=(-20,20)),
                                       transforms.Lambda(lambda img: img.point(lambda i: i*np.random.uniform(1.2,1.5)))])

data_train = datasets.DatasetFolder(root='data/chest_xray/train',loader = loader,extensions = ('jpeg'),
                                    transform=transformaciones)
data_test = datasets.DatasetFolder(root='data/chest_xray/test',loader = loader,extensions = ('jpeg'),
                                   transform=transformaciones)

parte 2

In [ ]:
num_1_train = 0
num_0_train = 0
for x in data_train:
    if x[1] == 0:
        num_0_train += 1
    else:
        num_1_train += 1

In [22]:
num_1_train,num_0_train

(3883, 1349)

In [10]:
num_1_test = 0
num_0_test = 0
for x in data_test:
    if x[1] == 0:
        num_0_test += 1
    else:
        num_1_test += 1

In [12]:
num_1_test,num_0_test

(390, 234)

parte 3

In [168]:
from sklearn.model_selection import train_test_split

X_tr = np.array([np.asarray(obs[0]) for obs in data_train])
y_tr = np.array([obs[1] for obs in data_train])

x_train, x_valid, y_train, y_valid = train_test_split(X_tr, y_tr, test_size = .2)

In [ ]:
class ReplicarMuestreoDePrueba(torch.utils.data.Sampler):
    def __init__(self, etiquetas_prueba, indices_val, etiquetas_val):
        self.etiquetas_prueba = etiquetas_prueba
        self.indices_val = indices_val
        self.etiquetas_val = etiquetas_val
        
    def __iter__(self):
        p_test = np.sum(self.etiquetas_prueba)/len(self.etiquetas_prueba) # prob de ser 1, en testeo
        if p_test < 0.5:
            p_vect = self.etiquetas_val 
        else:
            p_vect = 1 - self.etiquetas_val
        p_vect = p_vect/np.sum(p_vect) # debe sumar 1
        sampleo = self.indices_val
        sampleo_eti = self.etiquetas_val
        p_sampleo = np.sum(sampleo_eti)/len(sampleo_eti)
        while abs(p_sampleo - p_test) > 1e-3:
            index = np.random.choice(self.indices_val, size=1, p=p_vect)
            sampleo = np.append(sampleo, index)
            sampleo_eti = np.append(sampleo_eti, self.etiquetas_val[index])
            p_sampleo = np.sum(sampleo_eti)/len(sampleo_eti)
        return sampleo


parte 4

In [ ]:
from torch.utils.data import DataLoader